In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\torch\utils\tensorboard\__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\stable_baselines3\common\cmd_util.py:5: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  warnings.warn(


In [2]:
env = make_vec_env("G1GoalDist-v1", 32)

c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\envs\torch\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MlpPolicy", 
    env, 
    verbose=1,
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=2,
    ),
    batch_size=2048,
    # buffer_size=14000000,
    learning_rate=1e-4,
    tensorboard_log="./logs/sac/"
)

Using cuda device


MemoryError: Unable to allocate 12.6 GiB for an array with shape (437500, 32, 121) and data type float64

In [ ]:
# model.set_parameters("trained_models/g1-reptile/sac")

In [ ]:
model.learn(
    total_timesteps=int(1e7), 
    log_interval=100,
    tb_log_name='reptile',
    reset_num_timesteps=False,
)

In [ ]:
model.save("trained_models/g1-reptile/sac")